# Credit Card Fraud Detection with Autoencoders!

## Problem Statement

## Imports

In [56]:
import pdb
import sys
import os
import time
import pandas as pd
from sklearn.metrics import f1_score
import pathlib
import copy
import pickle
import warnings
from datetime import datetime
from sklearn.manifold import TSNE

import matplotlib.pyplot as plt
from matplotlib import cm
import seaborn as sns
from matplotlib import style
style.use('seaborn')

import cv2

import numpy as np
import pandas as pd

import torch
from torch.autograd import Variable
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import Sampler
import torch.nn as nn

In [27]:
%matplotlib inline
pd.options.display.float_format = '{:,.2f}'.format
seed = 5
np.random.seed(seed)
torch.manual_seed(seed)

# Getting the Dataset

In [23]:
PATH = pathlib.Path('../input/'); list(PATH.iterdir())

[PosixPath('../input/creditcard.csv')]

In [22]:
list((PATH/'input').iterdir())

[PosixPath('../input/creditcard.csv')]

In [25]:
df = pd.read_csv(list(PATH.iterdir())[0]); df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,-0.551600,-0.617801,-0.991390,-0.311169,1.468177,-0.470401,0.207971,0.025791,0.403993,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,1.612727,1.065235,0.489095,-0.143772,0.635558,0.463917,-0.114805,-0.183361,-0.145783,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,0.624501,0.066084,0.717293,-0.165946,2.345865,-2.890083,1.109969,-0.121359,-2.261857,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,-0.226487,0.178228,0.507757,-0.287924,-0.631418,-1.059647,-0.684093,1.965775,-1.232622,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,-0.822843,0.538196,1.345852,-1.119670,0.175121,-0.451449,-0.237033,-0.038195,0.803487,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [45]:
df.shape

(284807, 32)

In [28]:
df.describe()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
count,"284,807.00","284,807.00","284,807.00","284,807.00","284,807.00","284,807.00","284,807.00","284,807.00","284,807.00","284,807.00","284,807.00","284,807.00","284,807.00","284,807.00","284,807.00","284,807.00","284,807.00","284,807.00","284,807.00","284,807.00","284,807.00","284,807.00","284,807.00","284,807.00","284,807.00","284,807.00","284,807.00","284,807.00","284,807.00","284,807.00","284,807.00"
mean,"94,813.86",0.00,0.00,-0.00,0.00,-0.00,0.00,-0.00,-0.00,-0.00,0.00,0.00,-0.00,0.00,0.00,0.00,0.00,-0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-0.00,-0.00,88.35,0.00
std,"47,488.15",1.96,1.65,1.52,1.42,1.38,1.33,1.24,1.19,1.10,1.09,1.02,1.00,1.00,0.96,0.92,0.88,0.85,0.84,0.81,0.77,0.73,0.73,0.62,0.61,0.52,0.48,0.40,0.33,250.12,0.04
min,0.00,-56.41,-72.72,-48.33,-5.68,-113.74,-26.16,-43.56,-73.22,-13.43,-24.59,-4.80,-18.68,-5.79,-19.21,-4.50,-14.13,-25.16,-9.50,-7.21,-54.50,-34.83,-10.93,-44.81,-2.84,-10.30,-2.60,-22.57,-15.43,0.00,0.00
25%,"54,201.50",-0.92,-0.60,-0.89,-0.85,-0.69,-0.77,-0.55,-0.21,-0.64,-0.54,-0.76,-0.41,-0.65,-0.43,-0.58,-0.47,-0.48,-0.50,-0.46,-0.21,-0.23,-0.54,-0.16,-0.35,-0.32,-0.33,-0.07,-0.05,5.60,0.00
50%,"84,692.00",0.02,0.07,0.18,-0.02,-0.05,-0.27,0.04,0.02,-0.05,-0.09,-0.03,0.14,-0.01,0.05,0.05,0.07,-0.07,-0.00,0.00,-0.06,-0.03,0.01,-0.01,0.04,0.02,-0.05,0.00,0.01,22.00,0.00
75%,"139,320.50",1.32,0.80,1.03,0.74,0.61,0.40,0.57,0.33,0.60,0.45,0.74,0.62,0.66,0.49,0.65,0.52,0.40,0.50,0.46,0.13,0.19,0.53,0.15,0.44,0.35,0.24,0.09,0.08,77.16,0.00
max,"172,792.00",2.45,22.06,9.38,16.88,34.80,73.30,120.59,20.01,15.59,23.75,12.02,7.85,7.13,10.53,8.88,17.32,9.25,5.04,5.59,39.42,27.20,10.50,22.53,4.58,7.52,3.52,31.61,33.85,"25,691.16",1.00


In [46]:
df.columns

Index(['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10',
       'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20',
       'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount',
       'Class', 'Day_hrs'],
      dtype='object')

In [55]:
class_df = df['Class'].value_counts().to_frame().reset_index()
class_df['% of total'] = class_df['Class'] / len(df) * 100
class_df.rename(columns={'index': 'Class', 'Class': 'Count'})

,Class,Count,% of total
0,0,284315,99.83
1,1,492,0.17


In [40]:
hours = df.describe()['Time']['max'] / (60*60)
print(f"This dataset spans {hours:.0f} hours in total")

This dataset spans 48 hours in total


In [43]:
df['Day_hrs'] = df['Time'] / (60*60) % 24

0   0.00
1   0.00
2   0.00
3   0.00
4   0.00
Name: Day_hrs, dtype: float64

## Exploratory Data Analysis

Visualise data with t-SNE and/or PCA

In [69]:
n = 1600
normal_smpl = df[df['Class'] == 0].sample(n=n)
fraud_smpl = df[df['Class'] == 1]

smpl = pd.concat((normal_smpl, fraud_smpl), axis=0).sample(frac=1).reset_index(drop=True)
inp = smpl.drop(columns=['Class']).values
targ = smpl['Class'].values

In [ ]:
def tsne_plot(inp, targ):
    tsne = TSNE(n_components=2, random_state=seed)
    train = tsne.fit_transform(inp)

    plt.figure(figsize=(10,8))
    plt.title("t-SNE visualisation by transaction type:")
    plt.scatter(inp[np.where(targ == 0), 0], inp[np.where(targ == 0), 1], marker='o', color='g', alpha=0.6, label="Normal")
    plt.scatter(inp[np.where(targ == 1), 0], inp[np.where(targ == 1), 1], marker='o', color='r', alpha=0.6, label="Fraud")
    plt.legend(loc='best')
    plt.show()

tsne_plot(inp, targ)

Traditional stats

Autoencoder

Classifier based on Autoencoder Error

Final stats